In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

2024-02-13 20:39:29.585982: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 20:39:29.721964: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 20:39:29.722017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 20:39:29.742968: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 20:39:29.785012: I tensorflow/core/platform/cpu_feature_guar

In [14]:


class grad_initilization:
    [tf.config.experimental.set_memory_growth(device, True) for device in tf.config.list_physical_devices('GPU')]
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    else:
        raise RuntimeError("No GPU devices found.")

    
    def __init__(self, loss_function='mse', physical_devices=0):
        self.loss_function = loss_function
        self.initial_prediction = None
        self.physical_devices = physical_devices
        
    ### Find a good starting point for point. ie. initializing the gradient boosting alo
    def target_mean(self, y):
        return np.mean(y)

    ### MSE = The squared error between the predicted value and the actual value
    ### So when the algo guess guesses y for a given x, this measures how far off p is from y
    def mse(self, y, p):
        return np.mean((y - p)**2) 

    def startpoint_mse(self, y):
        """Initialize the gradient boosting model."""
        if self.loss_function == 'mse':
            self.initial_prediction = self.target_mean(y)
            print(f"Initial model prediction (mean of y): {self.initial_prediction}")
            initial_mse = self.mse(y, self.initial_prediction)
            print(f"Initial MSE: {initial_mse}")
            return self.initial_prediction
        else:
            raise ValueError("Unsupported loss function. Currently, only 'mse' is implemented.")
        


if __name__ == "__main__":
    n = 4096
    y = tf.random.uniform((n, n), dtype=float)
    
    # Initialize the gradient boosting model
    gbm_initializer = grad_initilization()
    gbm_initializer.startpoint_mse(y)

Initial model prediction (mean of y): 0.49999353289604187
Initial MSE: 0.08333720266819


In [7]:
import numpy as np

N = 4096  # Size of the matrices
A = np.random.rand(N, N)  # Create a N x N matrix A with random values
B = np.random.rand(N, N)  # Create a N x N matrix B with random values

C = A @ B  # Perform matrix multiplication
# Due to the large size, it's not practical to print C. You might want to perform further operations here.
